In [1]:
!pip install dotenv


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import requests
from pprint import pprint
from dotenv import load_dotenv

# Load environment variables from a .env file
load_dotenv()

# Retrieve the VEYRAX_API_KEY from the environment variables
api_key = os.getenv("VEYRAX_API_KEY")
if not api_key:
    raise ValueError("The environment variable VEYRAX_API_KEY is not set.")

# Base URL for the veyrax.com API
# BASE_URL = "https://veyrax.com/api"  # Adjust the URL if needed
BASE_URL = "http://localhost:8000"

In [3]:
def call_veyrax_api(endpoint: str, method: str = "POST", params: dict = None, data: dict = None, json: dict = None) -> dict:
    """
    Call any endpoint of the veyrax.com API.

    Args:
        endpoint (str): The API endpoint (path after the base URL, e.g., '/tools').
        method (str): HTTP method to use ("GET", "POST", etc.).
        params (dict, optional): Query parameters for the request.
        data (dict, optional): Form data for the request.
        json (dict, optional): JSON payload for the request.

    Returns:
        dict: The JSON response from the API.
    """
    url = f"{BASE_URL}{endpoint}"
    
    # Set the header with the VEYRAX_API_KEY
    headers = {
        "VEYRAX_API_KEY": api_key
    }
    
    # Choose the method and make the request
    try:
        response = requests.request(method, url, headers=headers, params=params, data=data, json=json)
        response.raise_for_status()  # Raise an error for bad status codes
        return response.json()
    except requests.RequestException as e:
        print(f"Error calling API: {e}")
        return {"error": str(e)}


In [4]:
# Add your first integration, go to: https://veyrax.com/integrations

In [5]:
result = call_veyrax_api("/get-tools", method="GET")

In [6]:
# Your available tools:
print(result.get('tools', {}).keys())

dict_keys(['resend', 'notion', 'gmail', 'confluence', 'openweather', 'google-calendar', 'google-drive', 'dropbox', 'slack', 'perplexity', 'jira', 'outlook', 'linear'])


In [7]:
# Let's see available methods for Gmail
print(result.get('tools', {}).get('gmail', {}).keys())

dict_keys(['get_drafts', 'get_message', 'create_draft', 'delete_draft', 'get_messages', 'modify_draft', 'send_message', 'delete_message'])


In [8]:
# Let's see annotation for create_draft
print(result.get('tools', {}).get('gmail', {}).get('create_draft', {}))

{'type': 'function', 'function': {'name': '/gmail/create_draft', 'strict': True, 'parameters': {'type': 'object', '$defs': {'DraftMessage': {'type': 'object', 'title': 'DraftMessage', 'required': ['recipient', 'subject', 'body'], 'properties': {'body': {'type': 'string', 'title': 'Body'}, 'subject': {'type': 'string', 'title': 'Subject'}, 'recipient': {'type': 'string', 'title': 'Recipient'}}, 'additionalProperties': False}}, 'title': 'CreateDraftParams', 'required': ['draft_args'], 'properties': {'draft_args': {'$ref': '#/$defs/DraftMessage'}}, 'additionalProperties': False}, 'description': 'Handle POST /gmail/create_draft'}}


In [30]:
payload = {"draft_args" : {"recipient": "hello@veyrax.com", "subject": "Thank you for this demo!", "body": "Hello World!"}}
draft = call_veyrax_api('/tool-call/gmail/create_draft', json=payload)

In [31]:
draft.get('data')

{'id': '1955802086e58054',
 'thread_id': '1955802086e58054',
 'subject': 'Thank you for this demo!',
 'snippet': 'Hello World!',
 'body': 'Hello World!',
 'recipient': 'hello@veyrax.com',
 'sender': 'veyraxservice@gmail.com',
 'date': '2025-03-02T12:58:02-05:00',
 'html_link': 'https://mail.google.com/mail/u/0/#inbox/1955802086e58054',
 'labels': ['DRAFT'],
 'draft_id': 'r6493674990151330653'}

In [32]:
component = draft.get('component')

In [33]:
def render_dynamic_components_page(component):
    import json
    # Serialize the components list (each with keys: name, props, jsx)
    components_json = json.dumps(component)
    
    with open("dynamic_components.html", "r", encoding="utf-8") as f:
        template = f.read()
        
    return template.replace("{components_json}", components_json)

In [34]:
from IPython.display import IFrame

In [35]:
def render_and_show(component):
    html = render_dynamic_components_page(component)
    with open("component.html", "w") as f:
        f.write(html)
    return IFrame(src='component.html', width=400, height=200)

In [36]:
render_and_show(component)

In [37]:
# Let's see annotation for send_message
print(result.get('tools', {}).get('gmail', {}).get('send_message', {}))

{'type': 'function', 'function': {'name': '/gmail/send_message', 'strict': True, 'parameters': {'type': 'object', 'title': 'SendMessageParams', 'required': ['message_id'], 'properties': {'message_id': {'type': 'string', 'title': 'Message Id'}}, 'additionalProperties': False}, 'description': 'Handle POST /gmail/send_message'}}


In [38]:
send_message = call_veyrax_api('/tool-call/gmail/send_message', json={"message_id": draft.get('data', {}).get('id')})

In [39]:
# we sent a message:
render_and_show(send_message.get('component', {}))